# Using `numba.jit` to speedup the computation of the Cityblock distance matrix 


In this notebook we implement a function to compute the Cityblock distance matrix using Numba's parallelism and the *just-in-time* compilation decorator.

In [ ]:
import numpy as np
import numba

In [ ]:
@numba.njit()
def cityblock_numba(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


@numba.njit(parallel=True)
def cityblock_numba_parallel(x, y):
    """Implementation with numba."""

    num_samples, num_feat = x.shape
    dist_matrix = np.empty((num_samples, num_samples))
    for i in numba.prange(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in range(num_feat):
                r += np.abs(x[i][k] - y[j][k])
            dist_matrix[i][j] = r

    return dist_matrix


def cityblock_broadcast(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = (x_ij - y_ij)^2
    """
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]

    return np.abs(diff).sum(axis=2)

In [ ]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit cityblock_numba_parallel(x,x)
%timeit cityblock_numba(x,x)

In [ ]:
np.abs(cityblock_numba_parallel(x,x) - cityblock_broadcast(x, x)).sum()